In [30]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from tf_keras.layers import Dense
from tf_keras.models import Sequential
from tf_keras.callbacks import EarlyStopping

In [22]:
# inputs
#########
NAME = 'jaylen_brown'
model = LinearRegression()
#########

train = pd.read_csv(f"./player_game_logs/{NAME}/{NAME}_TRAIN.csv")
test = pd.read_csv(f"./player_game_logs/{NAME}/{NAME}_TEST.csv")

X_train = train.drop(columns=['PTS'])
X_test = test.drop(columns=['PTS'])
y_train = train['PTS']
y_test = test['PTS']
n_train = y_train.count()
n_test = y_test.count()
test_mean = np.mean(y_test)

# scale features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model.fit(X_train_scaled, y_train)

y_train_pred = model.predict(X_train_scaled)
mse_train = mean_squared_error(y_train, y_train_pred)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(y_train, y_train_pred)

y_pred_test = model.predict(X_test_scaled)
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test, y_pred_test)

rounded_preds = []
for p in y_pred_test:
    rounded_preds.append(round(p, 4))

y = list(zip(y_test, rounded_preds))

print(f"player: {NAME}\n")
print(f"(actual_pts, predicted_pts) pairs: \n{str(y)}\n")
print({f"{col}_weight": coef for col, coef in zip(X_train.columns, model.coef_)})
print(f"bias: {model.intercept_}")
print(f"y_test_mean: {test_mean}")
print(f"r2_train: {r2_train}")
print(f"r2_test: {r2_test}")

player: jaylen_brown

(actual_pts, predicted_pts) pairs: 
[(22, 20.9766), (24, 22.7832), (19, 28.2773), (19, 15.4707), (31, 22.8379), (12, 11.9531), (10, 9.168), (21, 28.6211), (17, 20.5527), (34, 19.6973), (16, 26.4961), (23, 25.6738), (21, 23.9805), (18, 14.4805), (4, 10.4824), (17, 23.8555), (25, 14.6797), (0, 7.9492), (13, 13.4688), (12, 12.4473), (8, 13.7852), (8, 15.0059), (18, 21.627), (10, 15.2656), (14, 15.8496), (19, 20.9336), (3, 6.3105), (17, 22.8262), (28, 23.6641), (22, 24.5859), (26, 19.0449), (23, 25.0586), (7, 17.2402), (8, 9.0879), (16, 15.0898), (32, 26.0977), (21, 23.5488), (12, 18.2852), (2, 10.4727), (6, 7.0156), (37, 22.4805), (9, 14.084), (12, 15.9238), (15, 5.9453), (35, 19.9727), (21, 25.6016), (10, 8.8887), (31, 27.1211), (34, 24.5449), (16, 26.3926), (10, 13.3145), (31, 16.8496), (26, 26.0), (5, 9.9355), (29, 28.6426), (3, 8.2754), (7, 6.0625), (21, 20.8184), (34, 21.8555), (21, 16.9629), (24, 17.6914), (11, 14.1348), (6, 17.5781), (9, 13.1035), (30, 26.0996

In [51]:
## Feedforward NN playground
NAME = 'danny_manning'
train = pd.read_csv(f"./player_game_logs/{NAME}/{NAME}_TRAIN.csv")
test = pd.read_csv(f"./player_game_logs/{NAME}/{NAME}_TEST.csv")

X_train = train.drop(columns=['PTS'])
X_test = test.drop(columns=['PTS'])
y_train = train['PTS']
y_test = test['PTS']
n_train = y_train.count()
n_test = y_test.count()
test_mean = np.mean(y_test)

scaler_X = MinMaxScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train.to_numpy().reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.to_numpy().reshape(-1, 1)).flatten()

model = Sequential([
    Dense(87, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(20, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# train the model
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
history = model.fit(X_train_scaled, y_train_scaled,
epochs=20,
batch_size=32,
#validation_split=0.2,
callbacks=[early_stopping]
)

train_predictions = model.predict(X_train_scaled)
test_predictions = model.predict(X_test_scaled)

# scale back the predictions to the original value range
y_train_pred = scaler_y.inverse_transform(train_predictions.reshape(-1, 1)).flatten()
y_test_pred = scaler_y.inverse_transform(test_predictions.reshape(-1, 1)).flatten()

mse_train = mean_squared_error(y_train, y_train_pred)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(y_train, y_train_pred)

mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test, y_test_pred)

rounded_preds = []
for p in y_test_pred:
    rounded_preds.append(round(p, 4))

y = list(zip(y_test, rounded_preds))

print(f"player: {NAME}\n")
print(f"(actual_pts, predicted_pts) pairs: \n{str(y)}\n")
print(f"rmse_train: {rmse_train}")
print(f"rmse_test: {rmse_test}")
print(f"r2_train: {r2_train}")
print(f"r2_test: {r2_test}")

Epoch 1/20
23/23 [==============================] - 2s 4ms/step - loss: 0.1652
Epoch 2/20
23/23 [==============================] - 0s 4ms/step - loss: 0.0328
Epoch 3/20
23/23 [==============================] - 0s 4ms/step - loss: 0.0253
Epoch 4/20
23/23 [==============================] - 0s 4ms/step - loss: 0.0222
Epoch 5/20
23/23 [==============================] - 0s 4ms/step - loss: 0.0203
Epoch 6/20
23/23 [==============================] - 0s 4ms/step - loss: 0.0193
Epoch 7/20
23/23 [==============================] - 0s 5ms/step - loss: 0.0181
Epoch 8/20
23/23 [==============================] - 0s 6ms/step - loss: 0.0176
Epoch 9/20
23/23 [==============================] - 0s 4ms/step - loss: 0.0171
Epoch 10/20
23/23 [==============================] - 0s 4ms/step - loss: 0.0166
Epoch 11/20
23/23 [==============================] - 0s 5ms/step - loss: 0.0159
Epoch 12/20
23/23 [==============================] - 0s 4ms/step - loss: 0.0164
Epoch 13/20
23/23 [==============================